In [1]:
import pandas as pd
import numpy as np

df_list = []
train_list = []
test_list = []
file_name = 'data/standard_{}.csv'
test_months = [2012, 2102, 2104, 2106, 2108, 2110, 2112]
months_list_start_1 = [1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111]
months_list_start_2 = [1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112]
month = pd.read_csv('data/standard_1901.csv', sep='|', dtype={'FOA': 'str', 'B-day': 'str', 'Rat': 'str'})
month['Xp'] = 1
month['RatDiff'] = 0
month['RatChg'] = 0
month = month.drop(columns=['Name'], axis=1)
df_list.append(month)
train_list.append(month)
k = 1
for i, j in zip(months_list_start_1, months_list_start_2):
    month1 = df_list[-1]
    month1['FOA'] = pd.Series(month1['FOA'], dtype='str')
    month1['B-day'] = pd.Series(month1['B-day'], dtype='str')
    month1['Rat'] = pd.Series(month1['Rat'], dtype='str')
    month2 = pd.read_csv(file_name.format(j), sep='|', dtype={'FOA': 'str', 'B-day': 'str', 'Rat': 'str'})
    month1['Rat'] = month1['Rat'].str.replace('M ', '')
    month1['Rat'] = month1['Rat'].astype('int64')
    month2['Rat'] = month2['Rat'].str.replace('M ', '')
    month2['Rat'] = month2['Rat'].astype('int64')
    prev_rating = month1[['ID Number', 'Rat', 'Xp']]
    prev_rating = prev_rating.rename(columns={"Rat": "PrevRat"})
    month2 = pd.merge(month2, prev_rating, how='left', sort=False)
    month2['Xp'] = np.where(np.isnan(month2['PrevRat']), 0, k)
    k += 1
    month2['Xp'] = month2['Xp'] + 1
    month2['PrevRat'] = month2['PrevRat'].fillna(month2['Rat'])
    month2['RatDiff'] = month2['Rat'] - month2['PrevRat']
    month2['RatChg'] = np.where(month2['RatDiff'] == 0, 0, np.where(month2['RatDiff'] >  0, 1, 2))
    month2 = month2.drop(columns=['PrevRat', 'Name'], axis=1)
    df_list.append(month2)
    if j in test_months:
        test_list.append(month2)
    else:
        train_list.append(month2)

df = pd.concat(df_list)
df.columns = df.columns.str.lower()
df['b-day'] = df['b-day'].str.replace('0 ', '')
df['b-day'] = df['b-day'].astype('int64')
df['b-day'].replace(to_replace = 0, value = df['b-day'].median(), inplace=True)
df['age'] = df['year'] - df['b-day']
df = df.drop(columns=['b-day'], axis=1)

train = pd.concat(train_list)
test = pd.concat(test_list)

train.columns = train.columns.str.lower()
train['b-day'] = train['b-day'].str.replace('0 ', '')
train['b-day'] = train['b-day'].astype('int64')
train['b-day'].replace(to_replace = 0, value = train['b-day'].median(), inplace=True)
train['age'] = train['year'] - train['b-day']
train = train.drop(columns=['b-day'], axis=1)

test.columns = test.columns.str.lower()
test['b-day'] = test['b-day'].str.replace('0 ', '')
test['b-day'] = test['b-day'].astype('int64')
test['b-day'].replace(to_replace = 0, value = test['b-day'].median(), inplace=True)
test['age'] = test['year'] - test['b-day']
test = test.drop(columns=['b-day'], axis=1)

compression_opts = dict(method='zip', archive_name='data/full.csv')
df.to_csv('data/full.zip', index=False, compression=compression_opts)

compression_opts = dict(method='zip', archive_name='data/train.csv')
train.to_csv('data/train.zip', index=False, compression=compression_opts)

compression_opts = dict(method='zip', archive_name='data/test.csv')
test.to_csv('data/test.zip', index=False, compression=compression_opts)

In [2]:
train = pd.read_csv('train.zip')
test = pd.read_csv('test.zip')

In [3]:
train['tit'] = train['tit'].fillna('NT')
train['foa'] = train['foa'].fillna('NT')
test['tit'] = test['tit'].fillna('NT')
test['foa'] = test['foa'].fillna('NT')
train = train.drop(columns=['wtit', 'otit', 'flag'], axis=1)
test = test.drop(columns=['wtit', 'otit', 'flag'], axis=1)

In [8]:
train['foa'] = train['foa'].str.replace(',IO', 'IO')
train['foa'] = train['foa'].str.replace('FTIO', 'FT,IO')
train['foa'] = train['foa'].str.replace('FI    AIM', 'FI,AIM')
train['foa'] = train['foa'].str.replace('IA    AGM', 'IA,AGM')

In [9]:
train['tit'] = train['tit'].fillna('NT')
test['tit'] = test['tit'].fillna('NT')

In [10]:
train['k'].replace([2, 10, 15, 20, 40],
                        [0, 1, 2, 3, 4], inplace=True)

In [11]:
test['k'].replace([2, 10, 15, 20, 40],
                        [0, 1, 2, 3, 4], inplace=True)

In [12]:
train = train.drop(columns=['year', 'mon'], axis=1)
test = test.drop(columns=['year', 'mon'], axis=1)

In [13]:
train['fed'].replace(['IND','BAN','MAS','CHN','IRQ','BRN','INA','GER','IRI','SWE','NOR','DEN','SRI','SGP','TAN','USA','SCO','NED','FIN','ESP','FRA','MAR','ENG','EGY','JOR','TUR','CUB','PER','ECU','DOM','URU','RUS','SUI','KGZ','PLE','PHI','GEO','UKR','ARG','ISR','BOL','CHI','MEX','CRC','ESA','AUS','ETH','AZE','TUN','VEN','ITA','CYP','GRE','KAZ','CRO','LBN','JAM','CAN','SYR','BEL','ALG','UZB','BRA','MTN','LBA','SUD','POL','UAE','YEM','BIH','SOM','PAR','SUR','TKM','MDV','QAT','AFG','PAK','NGR','TJK','ROU','POR','SLO','JPN','ANG','HUN','SRB','SVK','MLT','MAC','CZE','ARM','BRU','ZAM','EST','AUT','FID','LTU','LAT','TTO','KSA','LBR','RSA','NZL','MDA','BLR','MNE','MOZ','COL','KUW','KEN','NCA','SEY','GUA','PUR','MRI','NEP','HKG','LUX','GHA','ISV','PLW','ISL','ERI','WLS','KOS','SLE','ALB','FIJ','MGL','HAI','BUL','TOG','CMR','PAN','SOL','IRL','DJI','KOR','UGA','MKD','CIV','OMA','BOT','SSD','THA','ZIM','AND','MLI','GUY','MAD','BAR','HON','TLS','BER','VIE','FAI','MNC','ARU','MYA','TPE','JCI','ANT','SEN','GAM','LES','MAW','GUM','RWA','CPV','CGO','GCI','AHO','SMR','NAM','BUR','GAB','LAO','BHU','IVB','BDI','BAH','STP','NRU','SWZ','LIE','PNG','CAF','CAM','Ind','COD','LCA','CAY','IOM'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193], inplace=True)


In [14]:
test['fed'].replace(['IND','BAN','MAS','CHN','IRQ','BRN','INA','GER','IRI','SWE','NOR','DEN','SRI','SGP','TAN','USA','SCO','NED','FIN','ESP','FRA','MAR','ENG','EGY','JOR','TUR','CUB','PER','ECU','DOM','URU','RUS','SUI','KGZ','PLE','PHI','GEO','UKR','ARG','ISR','BOL','CHI','MEX','CRC','ESA','AUS','ETH','AZE','TUN','VEN','ITA','CYP','GRE','KAZ','CRO','LBN','JAM','CAN','SYR','BEL','ALG','UZB','BRA','MTN','LBA','SUD','POL','UAE','YEM','BIH','SOM','PAR','SUR','TKM','MDV','QAT','AFG','PAK','NGR','TJK','ROU','POR','SLO','JPN','ANG','HUN','SRB','SVK','MLT','MAC','CZE','ARM','BRU','ZAM','EST','AUT','FID','LTU','LAT','TTO','KSA','LBR','RSA','NZL','MDA','BLR','MNE','MOZ','COL','KUW','KEN','NCA','SEY','GUA','PUR','MRI','NEP','HKG','LUX','GHA','ISV','PLW','ISL','ERI','WLS','KOS','SLE','ALB','FIJ','MGL','HAI','BUL','TOG','CMR','PAN','SOL','IRL','DJI','KOR','UGA','MKD','CIV','OMA','BOT','SSD','THA','ZIM','AND','MLI','GUY','MAD','BAR','HON','TLS','BER','VIE','FAI','MNC','ARU','MYA','TPE','JCI','ANT','SEN','GAM','LES','MAW','GUM','RWA','CPV','GCI','AHO','SMR','NAM','BUR','GAB','LAO','BHU','IVB','BDI','BAH','STP','NRU','SWZ','LIE','PNG','CAF','CAM','Ind','COD','LCA','CAY','IOM'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193], inplace=True)


In [15]:
train['foa'].replace(['NT', 'ACM', 'AFM', 'AIM', 'AGM', 'FI,FA,IA,FT,IO,SIAGM', 'SI,SI', 'IA', 'FST', 'IO', 'FI,AIM', 'IO,FT', 'FT', 'SI', 'IA,AGM', 'IA,SI', 'FI', 'DI', 'SIAG'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], inplace=True)

In [16]:
test['foa'].replace(['NT', 'ACM', 'AFM', 'AIM', 'AGM', 'SIAG', 'SI'],
                        [0, 1, 2, 3, 4, 18, 13], inplace=True)

In [17]:
train['tit'].replace(['NT', 'WFM', 'IM', 'GM', 'WIM', 'FM', 'CM', 'WCM', 'WGM', 'WH'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)

In [18]:
test['tit'].replace(['NT', 'WFM', 'IM', 'GM', 'WIM', 'FM', 'CM', 'WCM', 'WGM', 'WH'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)

In [19]:
train['sex'].replace(['M', 'F'], [0, 1], inplace=True)
test['sex'].replace(['M', 'F'], [0, 1], inplace=True)

In [22]:
train = train.drop(columns=['id number'], axis=1)
test = test.drop(columns=['id number'], axis=1)

In [23]:
train = train.drop(columns=['ratdiff'], axis=1)
test = test.drop(columns=['ratdiff'], axis=1)

In [25]:
df['foa'] = df['foa'].str.replace(',IO', 'IO')
df['foa'] = df['foa'].str.replace('FTIO', 'FT,IO')
df['foa'] = df['foa'].str.replace('FI    AIM', 'FI,AIM')
df['foa'] = df['foa'].str.replace('IA    AGM', 'IA,AGM')
foa_type = pd.CategoricalDtype(categories=['NT', 'ACM', 'AFM', 'AIM', 'AGM', 'FI,FA,IA,FT,IO,SIAGM', 'SI,SI', 'IA', 'FST', 'IO', 'FI,AIM', 'IO,FT', 'FT', 'SI', 'IA,AGM', 'IA,SI', 'FI', 'DI', 'SIAG'], ordered=False)
df["foa"] = df["foa"].astype(foa_type)
df['foa'].replace(['NT', 'ACM', 'AFM', 'AIM', 'AGM', 'FI,FA,IA,FT,IO,SIAGM', 'SI,SI', 'IA', 'FST', 'IO', 'FI,AIM', 'IO,FT', 'FT', 'SI', 'IA,AGM', 'IA,SI', 'FI', 'DI', 'SIAG'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], inplace=True)
tit_type = pd.CategoricalDtype(categories=['NT', 'WFM', 'IM', 'GM', 'WIM', 'FM', 'CM', 'WCM', 'WGM', 'WH'], ordered=False)
df["tit"] = df["tit"].astype(tit_type)
df['tit'].replace(['NT', 'WFM', 'IM', 'GM', 'WIM', 'FM', 'CM', 'WCM', 'WGM', 'WH'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], inplace=True)
sex_type = pd.CategoricalDtype(categories=['M', 'F'], ordered=False)
df["sex"] = df["sex"].astype(sex_type)
df['sex'].replace(['M', 'F'], [0, 1], inplace=True)
fed_type = pd.CategoricalDtype(categories=['IND','BAN','MAS','CHN','IRQ','BRN','INA','GER','IRI','SWE','NOR','DEN','SRI','SGP','TAN','USA','SCO','NED','FIN','ESP','FRA','MAR','ENG','EGY','JOR','TUR','CUB','PER','ECU','DOM','URU','RUS','SUI','KGZ','PLE','PHI','GEO','UKR','ARG','ISR','BOL','CHI','MEX','CRC','ESA','AUS','ETH','AZE','TUN','VEN','ITA','CYP','GRE','KAZ','CRO','LBN','JAM','CAN','SYR','BEL','ALG','UZB','BRA','MTN','LBA','SUD','POL','UAE','YEM','BIH','SOM','PAR','SUR','TKM','MDV','QAT','AFG','PAK','NGR','TJK','ROU','POR','SLO','JPN','ANG','HUN','SRB','SVK','MLT','MAC','CZE','ARM','BRU','ZAM','EST','AUT','FID','LTU','LAT','TTO','KSA','LBR','RSA','NZL','MDA','BLR','MNE','MOZ','COL','KUW','KEN','NCA','SEY','GUA','PUR','MRI','NEP','HKG','LUX','GHA','ISV','PLW','ISL','ERI','WLS','KOS','SLE','ALB','FIJ','MGL','HAI','BUL','TOG','CMR','PAN','SOL','IRL','DJI','KOR','UGA','MKD','CIV','OMA','BOT','SSD','THA','ZIM','AND','MLI','GUY','MAD','BAR','HON','TLS','BER','VIE','FAI','MNC','ARU','MYA','TPE','JCI','ANT','SEN','GAM','LES','MAW','GUM','RWA','CPV','CGO','GCI','AHO','SMR','NAM','BUR','GAB','LAO','BHU','IVB','BDI','BAH','STP','NRU','SWZ','LIE','PNG','CAF','CAM','Ind','COD','LCA','CAY','IOM'], ordered=False)
df["fed"] = df["fed"].astype(fed_type)
df['fed'].replace(['IND','BAN','MAS','CHN','IRQ','BRN','INA','GER','IRI','SWE','NOR','DEN','SRI','SGP','TAN','USA','SCO','NED','FIN','ESP','FRA','MAR','ENG','EGY','JOR','TUR','CUB','PER','ECU','DOM','URU','RUS','SUI','KGZ','PLE','PHI','GEO','UKR','ARG','ISR','BOL','CHI','MEX','CRC','ESA','AUS','ETH','AZE','TUN','VEN','ITA','CYP','GRE','KAZ','CRO','LBN','JAM','CAN','SYR','BEL','ALG','UZB','BRA','MTN','LBA','SUD','POL','UAE','YEM','BIH','SOM','PAR','SUR','TKM','MDV','QAT','AFG','PAK','NGR','TJK','ROU','POR','SLO','JPN','ANG','HUN','SRB','SVK','MLT','MAC','CZE','ARM','BRU','ZAM','EST','AUT','FID','LTU','LAT','TTO','KSA','LBR','RSA','NZL','MDA','BLR','MNE','MOZ','COL','KUW','KEN','NCA','SEY','GUA','PUR','MRI','NEP','HKG','LUX','GHA','ISV','PLW','ISL','ERI','WLS','KOS','SLE','ALB','FIJ','MGL','HAI','BUL','TOG','CMR','PAN','SOL','IRL','DJI','KOR','UGA','MKD','CIV','OMA','BOT','SSD','THA','ZIM','AND','MLI','GUY','MAD','BAR','HON','TLS','BER','VIE','FAI','MNC','ARU','MYA','TPE','JCI','ANT','SEN','GAM','LES','MAW','GUM','RWA','CPV','CGO','GCI','AHO','SMR','NAM','BUR','GAB','LAO','BHU','IVB','BDI','BAH','STP','NRU','SWZ','LIE','PNG','CAF','CAM','Ind','COD','LCA','CAY','IOM'],
                        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193], inplace=True)
df = df.drop(columns=['wtit', 'otit', 'flag', 'id number', 'ratdiff', 'mon', 'year'], axis=1)


In [35]:
compression_opts = dict(method='zip', archive_name='data/train_prepared.csv')
train.to_csv('data/train_prepared.zip', index=False, compression=compression_opts)

compression_opts = dict(method='zip', archive_name='data/test_prepared.csv')
test.to_csv('data/test_prepared.zip', index=False, compression=compression_opts)

compression_opts = dict(method='zip', archive_name='full_prepared.csv')
df.to_csv('data/full_prepared.zip', index=False, compression=compression_opts)